# PyTorch basics and Linear Regression
### Back -propogation : Differences between Gradient descent, Stochiastic gradient descent and Batch gradient descent with examples

## PyTorch basics

In [179]:
# Opt for Auto complete while coding
import rlcompleter, readline
readline.parse_and_bind('tab:complete')

In [180]:
import torch

In [2]:
mat = torch.tensor([[[2,3],[3,4]] , [[2,5],[4,7]]])

In [3]:
mat

tensor([[[2, 3],
         [3, 4]],

        [[2, 5],
         [4, 7]]])

In [51]:
import numpy as np
ar1 = np.array([[[2,4],[3,5]],[[6,3],[1,2]]])
ar = np.array([[[2,4],[3,5]],[[6,3],[1,2,6]]]) #This doesnt give error even though the last matrix has 3 elements


In [5]:
mat = torch.tensor([[[2,4],[3,5]],[[6,3],[1,2,6]]]) #This  gives error

ValueError: expected sequence of length 2 at dim 2 (got 3)

In [54]:
w = torch.tensor(3., requires_grad=True)
u = torch.tensor(4., requires_grad=True)

In [58]:
y = u + w
print(y)
y.backward()
#derivative
print("dy/dw = " , w.grad )

tensor(7., grad_fn=<AddBackward0>)


In [64]:
x = np.array([1.,2.])
z = torch.tensor(x)  #convert Array to tensor
print(type(z))
y = torch.from_numpy(x) #convert Array to tensor
print(type(y))
m = y.numpy()  #convert tensor to ndarray
print(type(m))

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [65]:
y.is_cuda # This tensor is not stored in GPu, it is stored in CPU

False

In [73]:
torch.tensor(3, dtype=torch.float32)
#torch.tensor(3, dtype='float32')#gives error
p = np.array(3, dtype='float32')

In [82]:
x1 = torch.tensor([1,2])
x2 = torch.tensor([3,1])
z = x1 @ x2
print("Matrix multiplication = dot_product using @",z)

p = x1*x2
print("Elemet-wise multiplication using *",p)

q = torch.dot(x1,x2)
print("Matrix multiplication = dot_product using dot method", q)


Matrix multiplication = dot_product using @ tensor(5)
Elemet-wise multiplication using * tensor([3, 2])
Matrix multiplication = dot_product using dot method tensor(5)


## Linear Regression using basic tensor operations

In [133]:
# 5 examples, each example gives Temp,rainfall,humidity values. Eg: [73(temp),67(rainfall),43(humidity)]
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
print(inputs.shape)
x = torch.from_numpy(inputs) #convert to tensor

(5, 3)


In [134]:
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')
print(targets.shape)
y = torch.from_numpy(targets)
print("Range of target ",np.min(targets)," - " , np.max(targets))

(5, 2)
Range of target  22.0  -  133.0


In [135]:
#Initialise weight matrix with random values. Values picked from Normal distribution, usually ranging from -1 to +1, mean=0 and std deviation =1
torch.manual_seed(3)
w =torch.randn(2,3, requires_grad=True)
#w = torch.empty((1,3)).normal_(mean=0,std=1) #Other way of initialising the weights
b =torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.8033,  0.1748,  0.0890],
        [-0.6137,  0.0462, -1.3683]], requires_grad=True)
tensor([0.3375, 1.0111], requires_grad=True)


In [136]:
def model(x):
    yhat = x @ w.t() + b
    return yhat

In [137]:
yhat = model(x)
print("Predicted matrix ", yhat)
print("Actual target ", y)

Predicted matrix  tensor([[  74.5165,  -99.5312],
        [  94.5155, -138.3418],
        [  98.8109, -125.5529],
        [  93.0817, -110.2279],
        [  78.7760, -132.6801]], grad_fn=<AddBackward0>)
Actual target  tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [138]:
#Function : Cost function mse 
def loss_mse(y,yhat):
    diff = y- yhat
    sqr = diff*diff  # * -> elementwise multiplication
    mse = torch.sum(sqr)/sqr.numel()
    return mse

### INITIAL LOSS BEFORE APPLYING GRADIET DESCENT

In [139]:
print("INITIAL LOSS BEFORE APPLYING GRADIET DESCENT")
loss = loss_mse(y,yhat)
print("Average Loss", loss)
elementwise_loss = torch.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss)

INITIAL LOSS BEFORE APPLYING GRADIET DESCENT
Average Loss tensor(24446.6367, grad_fn=<DivBackward0>)
Average loss in each element  tensor(156.3542, grad_fn=<SqrtBackward>)
On average, each element in the prediction differs from the actual target by about tensor(156.3542, grad_fn=<SqrtBackward>)


### Compute gradients - 1 ITERATION/1 EPOCH

In [140]:
loss.backward()

In [141]:
print(w)
wgrad = w.grad
print(wgrad)
# Inference
# For every unit change in w11, loss increases by 1280.80. To reduce the loss, we need to subtract w11 by 1280.80
# For every unit change in w12, loss increases by 91.129
# For every unit change in w21, loss decreased by 17806.6. To reduce the loss, we need to add w21 with 17806.6

tensor([[ 0.8033,  0.1748,  0.0890],
        [-0.6137,  0.0462, -1.3683]], requires_grad=True)
tensor([[  1280.8098,     91.1297,    284.9165],
        [-17806.6328, -19511.7695, -12133.7656]])


In [142]:
print(b)
bgrad = b.grad
print(bgrad)
# Inference
# For every 1 unit increase in b1, loss increases by 11.74
# For every 1 unit increase in b1, loss decreases by -213.2

tensor([0.3375, 1.0111], requires_grad=True)
tensor([  11.7401, -213.2668])


In [116]:
#Update the weights and biases matrix
# While updating the weights, we should not modify the gradients.Hence use, torch.no_grad
# While updating, we multiply grad with small number(learning rate - alpha), to ensure that we dont modify weights by large number.

with torch.no_grad():
    print("Initial weights",w)
    print("Initial bias",b)
    w-=wgrad * 1e-5
    b-=bgrad * 1e-5
    print("Updated weights ", w)
    print("Updated bias matrix", b)
    wgrad.zero_()
    bgrad.zero_()
    

Initial weights tensor([[ 0.8033,  0.1748,  0.0890],
        [-0.6137,  0.0462, -1.3683]], requires_grad=True)
Initial bias tensor([0.3375, 1.0111], requires_grad=True)
Updated weights  tensor([[ 0.7905,  0.1739,  0.0861],
        [-0.4357,  0.2413, -1.2469]], requires_grad=True)
Updated bias matrix tensor([0.3374, 1.0132], requires_grad=True)


### LOSS AFTER PERFORMING BACK_PROPOGATION 

In [117]:
# Calculate loss with these new weights and biases
yhat = model(x)
print("Predicted output",yhat)
loss = loss_mse(y,yhat)
print("Average Loss", loss)
elementwise_loss = torch.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss)

Predicted output tensor([[ 73.3979, -68.2398],
        [ 93.0874, -97.1996],
        [ 97.4091, -76.8757],
        [ 91.6305, -79.1834],
        [ 77.6052, -93.1664]], grad_fn=<AddBackward0>)
Average Loss tensor(16736.2539, grad_fn=<DivBackward0>)
Average loss in each element  tensor(129.3687, grad_fn=<SqrtBackward>)
On average, each element in the prediction differs from the actual target by about tensor(129.3687, grad_fn=<SqrtBackward>)


### RESULT INTERPRETATION after applying GRADIENT DESCENT/BACK-PROPOGATION ONCE
After first iteration loss reduced from 24446.6367 to 16736.255

After first iteration loss exhibited by each element reduced from 156.34 to 129.36, i.e each predicted element differs from actual value by 129.36 on an avg

1st iteration -> Takes all the 5 examples(x), calculates yhat, loss, grads and updates the weights. 
2nd iterations -> Takes all the 5 examples(x), calculates yhat, loss, grads and updates the weights.

## Gradient descent : Consider all examples at each epoch
### PERFOM BACK_PROPOGATION 100 times i.e epochs=100, Considering all examples in each epoch 
100 ITERATIONS = 100 EPOCHS , CONSIDERING ALL EXAMPLES IN EACH EPOCH

In [143]:
# Train for 100 epoch / say 100 iterations
# Hyperparameters ::  number of epoch(here,we assigned 100) and learning rate(here, we assigned 1e-5)
for i in range(100):
    yhat = model(x)
    loss = loss_mse(y,yhat)
    if(i % 10==0):
        print("Loss at iter " ,i ," = ",loss)
    loss.backward()
    with torch.no_grad():
        w-=w.grad * 1e-5
        b-=b.grad * 1e-5
        w.grad.zero_()  #If this is not done, for second iteration , it would take double derivative - f'(f'(dl/dw)) or f''(dl/dw)
        b.grad.zero_()

Loss at iter  0  =  tensor(24446.6367, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(1017.9881, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(671.6713, grad_fn=<DivBackward0>)
Loss at iter  30  =  tensor(601.9100, grad_fn=<DivBackward0>)
Loss at iter  40  =  tensor(544.7454, grad_fn=<DivBackward0>)
Loss at iter  50  =  tensor(494.2185, grad_fn=<DivBackward0>)
Loss at iter  60  =  tensor(449.4562, grad_fn=<DivBackward0>)
Loss at iter  70  =  tensor(409.7740, grad_fn=<DivBackward0>)
Loss at iter  80  =  tensor(374.5703, grad_fn=<DivBackward0>)
Loss at iter  90  =  tensor(343.3152, grad_fn=<DivBackward0>)


In [144]:
print("GRADIENT DESCENT ON 100 EPOCHS")
print("Loss at 99th iter", loss)
elementwise_loss = torch.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())

GRADIENT DESCENT ON 100 EPOCHS
Loss at 99th iter tensor(318.1753, grad_fn=<DivBackward0>)
Average loss in each element  tensor(17.8375, grad_fn=<SqrtBackward>)
On average, each element in the prediction differs from the actual target by about 17.837467193603516


## Stochiastic Gradient descent : 1 example at a time
### Basic tensor operations for back-propgation

In [127]:
for i in range(100):
    #print("At iteration ",i)
    for j in range(len(x)):
        yhat = model(x[j])
        #print(yhat)
        loss = loss_mse(y[j],yhat)
        #print(y[j])
        if(i % 10==0):
            print("Loss at iter " ,i ," = ",loss)
        loss.backward()
        with torch.no_grad():
            w-=w.grad * 1e-5
            b-=b.grad * 1e-5
            w.grad.zero_()  #If this is not done, for second iteration , it would take double derivative - f'(f'(dl/dw)) or f''(dl/dw)
            b.grad.zero_()



Loss at iter  0  =  tensor(1.6428, grad_fn=<DivBackward0>)
Loss at iter  0  =  tensor(2.5928, grad_fn=<DivBackward0>)
Loss at iter  0  =  tensor(9.1601, grad_fn=<DivBackward0>)
Loss at iter  0  =  tensor(0.7979, grad_fn=<DivBackward0>)
Loss at iter  0  =  tensor(13.0677, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(1.6126, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(2.4272, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(8.2193, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(0.7543, grad_fn=<DivBackward0>)
Loss at iter  10  =  tensor(11.8650, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(1.5844, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(2.2776, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(7.3740, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(0.7157, grad_fn=<DivBackward0>)
Loss at iter  20  =  tensor(10.7803, grad_fn=<DivBackward0>)
Loss at iter  30  =  tensor(1.5582, grad_fn=<DivBackward0>)
Loss at iter  30  =  tensor(2.1422, grad_f

In [128]:
print("STOCHIASTIC GRADIENT DESCENT RESULTS")
print("Loss at 99th iter", loss)
elementwise_loss = torch.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())

STOCHIASTIC GRADIENT DESCENT RESULTS
Loss at 99th iter tensor(5.2123, grad_fn=<DivBackward0>)
Average loss in each element  tensor(2.2831, grad_fn=<SqrtBackward>)
On average, each element in the prediction differs from the actual target by about 2.283052444458008


## Linear Regression using pytorch built-in functions -> Neural network lib(nn)

In [145]:
import torch.nn as nn

In [146]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [72, 66, 42], 
                   [90, 87, 65], [85, 134, 56], [100, 42, 33], 
                   [68, 95, 74], [63, 77, 53], [81, 48, 54], 
                   [57, 124, 48], [142, 43, 37], [67, 96, 80],[50,70,30]],dtype='float32')
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [66, 70], 
                    [84, 101], [120, 133], [22, 47], 
                    [103, 139], [59, 70], [81, 151], 
                    [129, 135], [23, 39], [104, 159],[50,60]], 
                   dtype='float32')
x = torch.from_numpy(inputs)
y = torch.from_numpy(targets)
x[[1,3]]

tensor([[ 91.,  88.,  64.],
        [102.,  43.,  37.]])

16 examples -> create batches 

In [147]:
# TensorDataset -> give input as output as tuple. Useful for taking small samples of data
from torch.utils.data import TensorDataset
train_ds = TensorDataset(x,y)
train_ds[:3]
#train_ds[[1,3]] #1st tensor is input, 2nd tensor is the target(y)

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [148]:
from torch.utils.data import DataLoader

In [150]:
#? DataLoader

In [151]:
batch_size = 4
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [152]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 50.,  70.,  30.],
        [100.,  42.,  33.],
        [ 63.,  77.,  53.]])
tensor([[56., 70.],
        [50., 60.],
        [22., 47.],
        [59., 70.]])


## 1 ITERATION, CONSIDERING ALL EXAMPLES - GRADIENT DESCENT
### Using pytorch in-built optimizer (SGD)

In [165]:
import torch.nn.functional as F
torch.manual_seed(3)
model = nn.Linear(3,2)
#print("Weight :: ",model.weight)
#print("Bias :: ",model.bias)
loss_fn =  F.mse_loss
opt = torch.optim.SGD(list(model.parameters()), lr=1e-5)

In [167]:
print(list(model.parameters())) # weights and bias matrices are the parameters
yhat = model(x)
loss = loss_fn(yhat,y)
loss.backward()
opt.step()
opt.zero_grad()
print("Loss",loss.item())
elementwise_loss = np.sqrt(loss.item())
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())
#print(list(model.parameters())) #UPDATED WEIGHTS AND BIASES AFTER OPTIMISATION

[Parameter containing:
tensor([[-0.4348, -0.3015, -0.1514],
        [-0.4091,  0.1158, -0.4122]], requires_grad=True), Parameter containing:
tensor([0.3157, 0.2831], requires_grad=True)]
Loss 22216.19140625
Average loss in each element  149.05096915568848
On average, each element in the prediction differs from the actual target by about 149.05096915568848


## 100 ITERATIONS(EPOCHS), CONSIDERING ALL EXAMPLES - GRADIENT DESCENT

In [168]:
import torch.nn.functional as F
torch.manual_seed(2) #We do this before creating model, since, model initialises the w, b matrices randomly
model = nn.Linear(3,2)
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr= 1e-15)

In [182]:
# At each epoch -> 
# we consider all 16 examples , predict output, calculate loss , grad descent and update the weights. 
# Thus, at each epoch we update the weights and bias matrix just once.
# With 100 epoch we will have 100 *1 = 100 losses calculated i.e weights will be updated 100 times (counter)


def gradDes(model, opt,loss_fn, x, epochs,counter=0):
    for i in range(epochs):
        counter=counter+1
        # model would have initialised w,b matrices. Predict the output considering input(X) and initialised w,b
        pred =yhat= model(x)
        #Find the loss from defined mse loss function. Pass actual and predicted values
        loss = loss_fn(yhat,y)
        #print(loss)
        # Perform backward/grad_des/differentiation on loss function
        loss.backward()
        
        #Optimisation : Update the w and b based on on grad descent calculation. grad optimizer takes care of this
        opt.step()
        
        # set calculated grad values of w, b to zero
        opt.zero_grad()
        
        #if (i+1) % 10 == 0:
            #print("Loss at epoch {} is {:.4f}".format(i,loss.item()))
        
    return (loss.item(),counter)
        
        

In [170]:
loss,counter = gradDes(model,opt,loss_fn,x,100)
print("counter",counter)
print("Loss at 99th iter", loss)
elementwise_loss = np.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())

counter 100
Loss at 99th iter 6332.97216796875
Average loss in each element  79.57997341020383
On average, each element in the prediction differs from the actual target by about 79.57997341020383


## 100 ITERATIONS, CONSIDERING 1 EXAMPLE AT A TIME - STOCHIASTIC GRADIENT DESCENT

In [171]:
len(train_ds)

16

In [172]:
import torch.nn.functional as F
torch.manual_seed(2) #We do this before creating model, since, model initialises the w, b matrices randomly
model = nn.Linear(3,2)
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr= 1e-15)

In [173]:
# At each epoch -> 
# first we consider 1 example and update the weights.
# next consider 2nd example  and updated weights and calculate loss and update weights and so on
# This continus for all 16 examples.Thus each epoch will have 16 losses calculated
# With 100 epoch we will have 100 *16= 1600 losses calculated i.e weights will be updated 1600 times (counter)

def stochGradDes(model, opt,loss_fn, x, epochs,counter=0):
    for i in range(epochs):
        for j in range(len(train_ds)):
            counter= counter+1
        # model would have initialised w,b matrices. Predict the output considering input(X) and initialised w,b
            pred = yhat= model(x[j])
        
        #Find the loss from defined mse loss function. Pass actual and predicted values
            loss = loss_fn(yhat,y[j])
        
        # Perform backward/grad_des/differentiation on loss function
            loss.backward()
        
        #Optimisation : Update the w and b based on on grad descent calculation. grad optimizer takes care of this
            opt.step()
        
        # set calculated grad values of w, b to zero
            opt.zero_grad()

            #if (i+1) % 10 == 0:
                #print("Loss at epoch {} is {:.4f}".format(i,loss.item()))
        
    return (loss.item(),counter)
        

In [174]:
(loss,counter) = stochGradDes(model,opt,loss_fn,x,100)
print("Loss at 99th iter", loss)
print("Counter = ",counter)
elementwise_loss = np.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())

Loss at 99th iter 1989.31591796875
Counter =  1600
Average loss in each element  44.60174792503933
On average, each element in the prediction differs from the actual target by about 44.60174792503933


## 100 ITERATIONS, CONSIDERING 4 EXAMPLES AT A TIME - BATCH GRADIENT DESCENT

In [175]:
import torch.nn.functional as F
torch.manual_seed(2) #We do this before creating model, since, model initialises the w, b matrices randomly
model = nn.Linear(3,2)
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr= 1e-15)

In [181]:
# At each epoch -> 
# first we consider 4 examples, predict the outputs for the 4 examples, calculate overall loss(sum loss for all examples)
# and perform grad descent and update the weights.
# next consider next 4 examples calculate loss and updated weights  so on
# This continues for all 16 examples i.e 4 batches -> each batch containing 4 examples shuffled(train_dl).Thus each epoch will have 4 losses calculated
# With 100 epoch we will have 100 *4= 400 losses calculated i.e weights will be updated 400 times


def batchGradDes(model, opt,loss_fn, x, epochs,counter=0):
    for i in range(epochs):
        for xb,yb in train_dl:
            counter = counter+1
        # model would have initialised w,b matrices. Predict the output considering input(X) and initialised w,b
            pred = yhat= model(xb)
        #Find the loss from defined mse loss function. Pass actual and predicted values
            loss = loss_fn(yhat,yb)

        # Perform backward/grad_des/differentiation on loss function
            loss.backward()
        
        #Optimisation : Update the w and b based on on grad descent calculation. grad optimizer takes care of this
            opt.step()
        
        # set calculated grad values of w, b to zero
            opt.zero_grad()
        
            #if (i+1) % 10 == 0:
                #print("Loss at epoch {} is {:.4f}".format(i,loss.item()))
        
    return (loss.item(),counter)
        

In [177]:
(loss,counter) = batchGradDes(model,opt,loss_fn,x,100)
print("Loss at 99th iter", loss)
print("counter",counter)
elementwise_loss = np.sqrt(loss)
print("Average loss in each element ",elementwise_loss)
print("On average, each element in the prediction differs from the actual target by about", elementwise_loss.item())

Loss at 99th iter 4408.11865234375
counter 400
Average loss in each element  66.39366424850905
On average, each element in the prediction differs from the actual target by about 66.39366424850905


# RESULTS 

### 1 iteration, all examples
Overall Loss = 22216.19140625 . 
On average, each element in the prediction differs from the actual target by about 149.05096915568848


### 100 iterations all examples
Overall Loss at 99th iter 6332.97216796875 .
On average, each element in the prediction differs from the actual target by about 79.57997341020383

### 100 iterations 1 example at a time
Overall Loss at 99th iter 1989.31591796875 .
On average, each element in the prediction differs from the actual target by about 44.60174792503933

### 100 iterations with batch_size=4, i.e considering 4 examples at a time
Overall Loss at 99th iter 4408.11865234375.
On average, each element in the prediction differs from the actual target by about 66.39366424850905

